In [30]:
#imports
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst
import gdal
import ogr
import osr
import gdalnumeric
import gdalconst
import os
import glob
from shutil import rmtree

In [31]:
gdal.UseExceptions()    # Enable exceptions

In [32]:
#import dir
iDirname = 'F:\\sen2cor\\import\\'
# working dir
wDirname = 'F:\\sen2cor\\working\\'
#export dir
eDirname = 'F:\\sen2cor\\export\\'


In [33]:
#L2a filename and subset region in degrees
ccLat = 45.40;
ccLong = -75.67;
rocWidth = 0.1;
rocWin = [ccLong-rocWidth, ccLat+rocWidth, ccLong+rocWidth, ccLat-rocWidth]
L1name = "S2B_MSIL1C_20180824T154809_N0206_R054_T18TVR_20180824T210113.SAFE"

In [34]:
#make a work directory - this allows parallel processing by multiple script executions
workDirname  = wDirname+L1name[0:len(L1name)-5]
os.mkdir(workDirname) 

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'F:\\sen2cor\\working\\S2B_MSIL1C_20180824T154809_N0206_R054_T18TVR_20180824T210113'

In [ ]:
# run sen2cor and find the result file
cmd = "F:\\sen2cor\\Sen2Cor-02.08.00-win64\\L2A_Process.bat "+iDirname+L1name+" --output_dir "+workDirname+'\\'
os.system(cmd)


In [ ]:
L2name = glob.glob(workDirname+L1name[0:8]+'2A'+L1name[10:24]+'*.SAFE')
exportName = (L2name[0][len(wDirname):(len(L2name[0])-5)])
print(exportName)

In [ ]:
# Open L2a data 
ds = gdal.Open(L2name[0]+'\MTD_MSIL2A.xml')
#ds.GetMetadata('SUBDATASETS')

In [ ]:
# Open L2a 10m dataset and print info
f10m = gdal.Open('SENTINEL2_L2A:'+L2name[0]+'\MTD_MSIL2A.xml'+':10m:EPSG_32618')
#gdal.Info(f10m)


In [ ]:
# Open L2a 20m dataset and print info
f20m = gdal.Open('SENTINEL2_L2A:'+L2name[0]+'\MTD_MSIL2A.xml'+':20m:EPSG_32618')
#gdal.Info(f20m)

In [ ]:
# Open L2a 60m dataset and print info
f60m = gdal.Open('SENTINEL2_L2A:'+L2name[0]+'\MTD_MSIL2A.xml'+':60m:EPSG_32618')
#gdal.Info(f20m)

In [17]:
# Export 10m version of L2A product (no ancillary bands) and output file describing contents
gdal.Translate(workDirname+'b4.tif',f10m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b3.tif',f10m,bandList=[2],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b2.tif',f10m,bandList=[3],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b8.tif',f10m,bandList=[4],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b5.tif',f20m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b6.tif',f20m,bandList=[2],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b7.tif',f20m,bandList=[3],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b8a.tif',f20m,bandList=[4],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b11.tif',f20m,bandList=[5],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b12.tif',f20m,bandList=[6],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b1.tif',f60m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)
gdal.Translate(workDirname+'b9.tif',f60m,bandList=[1],projWin = rocWin, projWinSRS = 'EPSG:4236',xRes=10,yRes=10)

vrtOptions = gdal.BuildVRTOptions(separate=True)
gdal.BuildVRT(workDirname+'merged.vrt',[workDirname+'b1.tif',workDirname+'b2.tif',workDirname+'b3.tif',workDirname+'b4.tif', \
                                     workDirname+'b5.tif',workDirname+'b6.tif',workDirname+'b7.tif',workDirname+'b8.tif', \
                                     workDirname+'b8a.tif',workDirname+'b9.tif',workDirname+'b11.tif',workDirname+'b12.tif'] \
                                   ,options=vrtOptions)
gdal.Translate(eDirname+exportName+'.tif',workDirname+'merged.vrt')
gdal.Info(workDirname+'merged.vrt')


In [ ]:

#delete working directory
rmtree(workDirname)